In [35]:
import cv2
import numpy as np

In [36]:
network = cv2.dnn.readNet('yolov3-spp.cfg', 'yolov3-spp.weights')

In [37]:
clasess = []
with open('coco.names', 'r') as f:
    clasess = f.read().splitlines()

In [38]:
img = cv2.imread('tes.jpg')
height, width, _ = img.shape

In [39]:
blob = cv2.dnn.blobFromImage(img, 1/255, (416,416), (0,0,0), swapRB = True, crop = False)

In [40]:
network.setInput(blob)

In [41]:
output_layers_names = network.getUnconnectedOutLayersNames()
layerOutputs = network.forward(output_layers_names)

In [42]:
boxes = []
confidences = []
class_ids = []

In [43]:
for output in layerOutputs:
    for detection in output:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0]*width)
            center_y = int(detection[1]*height)
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            boxes.append([x,y,w,h])
            confidences.append(float(confidence))
            class_ids.append(class_id)


In [44]:
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0,255, size=(len(boxes), 3))

In [45]:
if len(boxes) > 0:
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(clasess[class_ids[i]])
        confidence = str(round(confidences[i], 2))
        color = colors[i]
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, label + " " + confidence, (x, y + 20), font, 2, (255,255,255), 2)

In [46]:
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
